In [1]:
# Import basic libraries
import pandas as pd
import numpy as np
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point 
import os 
from pathlib import Path
import sys
from tqdm import tqdm  

# Define project root based on notebook location
def notebook_find_project_root(current: Path, marker: str = ".git"):
    for parent in current.resolve().parents:
        if (parent / marker).exists():
            return parent
    return current.resolve()  # fallback

PROJECT_ROOT = notebook_find_project_root(Path.cwd())
RAW_DIR = PROJECT_ROOT / "data" / "raw"
INTERIM_DIR = PROJECT_ROOT / "data" / "interim"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
EXTERNAL_DIR = PROJECT_ROOT / "data" / "external"
IN_DIR = PROJECT_ROOT / "data" / "processed" / "INDONESIA"

In [2]:
# Load in excel file and define future save directory
xls_path = RAW_DIR / "IN_DENGUE" / "DENGUE MONTHLY DATA (Updated).xlsx"
# Read in Indonesia shapefile
in_shp = EXTERNAL_DIR / "in_shp" / "Simplify_IDN"
# Read in Indonesia shapefile
in_shp = gpd.read_file(in_shp)

In [ ]:
# List of ID_2 values you want to highlight
highlight_ids = [123, 125, 153, 204, 240, 379]  # example list

# Separate GeoDataFrames
highlighted = in_shp[in_shp['ID_2'].isin(highlight_ids)]
rest = in_shp[~in_shp['ID_2'].isin(highlight_ids)]

# Plotting
fig, ax = plt.subplots(figsize=(10, 10))
rest.plot(ax=ax, color='lightgrey', edgecolor='white')
highlighted.plot(ax=ax, color='red', edgecolor='black')

# Add labels (optional)
for x, y, label in zip(highlighted.geometry.centroid.x, 
                       highlighted.geometry.centroid.y, 
                       highlighted['ID_2']):
    ax.text(x, y, str(label), fontsize=9, ha='center', va='center', color='black')

ax.set_title("Highlighted ID_2 Areas")
ax.axis('off')
plt.show()

In [3]:
# Loop through files and process CSV files
for file in os.listdir(INTERIM_DIR):
    if file.endswith('.csv'):
        # Create the variable name based on the file name (without extension)
        dataframe_name = file.split('.')[0].lower()
        
        # Load the CSV file into a DataFrame
        file_path = os.path.join(INTERIM_DIR, file)
        df = pd.read_csv(file_path)
        
        # Assign the DataFrame to the variable name in globals
        globals()[dataframe_name] = df
        
        # Print out the name of the dataframe and check the dataframe content
        print(f'DataFrame created: {dataframe_name}') 

DataFrame created: dengue_dubious_rows
DataFrame created: dengue_normalized_discrepancies
DataFrame created: dmi
DataFrame created: dmi_east
DataFrame created: enso_sst
DataFrame created: era_chirps_id_1993_2023
DataFrame created: lulc_200_full


In [ ]:
import pandas as pd

era_1013 = pd.read_csv(r'D:\Projects\TMU\gee_dengue\zonal statistic\era_missing_2010_2013.csv')
era_1417 = pd.read_csv(r'D:\Projects\TMU\gee_dengue\zonal statistic\era_missing_2014_2017.csv')
era_18 = pd.read_csv(r'D:\Projects\TMU\gee_dengue\zonal statistic\era_missing_2018_2021.csv')
era_1821 = pd.read_csv(r'D:\Projects\TMU\gee_dengue\zonal statistic\era_missing_2018aug_2021.csv')
era_2223 = pd.read_csv(r'D:\Projects\TMU\gee_dengue\zonal statistic\era_2022_2023.csv')

In [ ]:
# Concatenate all dataframes
era_id_132 = pd.concat([era_1013, era_1417, era_18, era_1821, era_2223], ignore_index=True)

# # Drop rows with any null values
# era_id_132 = era_id_132.dropna()

In [ ]:
era_id_132['ID_2'].unique()

In [ ]:


# Find rows with null values in 'temperature_2m_MEAN'
null_temp_rows = era_chirps_id_1993_2023[era_chirps_id_1993_2023['temperature_2m_MEAN'].isnull()]
print(f"Number of rows with nulls in 'temperature_2m_MEAN': {len(null_temp_rows)}")
print(null_temp_rows)

# Find rows with null values in 'precipitation_MEAN'
null_precip_rows = era_chirps_id_1993_2023[era_chirps_id_1993_2023['precipitation_MEAN'].isnull()]
print(f"\nNumber of rows with nulls in 'precipitation_MEAN': {len(null_precip_rows)}")
print(null_precip_rows)

In [ ]:
null_temp_rows['ID_2'].unique()

In [ ]:
null_precip_rows['ID_2'].unique()    

In [ ]:
# First drop the 'Unnamed: 0' column from the era_chirps_id_1993_2023 DataFrame
era_chirps_id_1993_2023 = era_chirps_id_1993_2023.drop(columns=['Unnamed: 0'])
null_rows_era_chirps = era_chirps_id_1993_2023[era_chirps_id_1993_2023.isnull().any(axis=1)]
# Display rows with null values (if any)
print(f"Number of rows with nulls: {len(null_rows_era_chirps)}")
print(null_rows_era_chirps)

In [ ]:
null_rows_era_chirps

In [ ]:
# Filter rows where 'precipitation_MEAN' is null
null_precipitation_ids = era_chirps_id_1993_2023[era_chirps_id_1993_2023['temperature_2m_MEAN'].isnull()]['ID_2'].unique()

# Display the unique ID_2 values
print("ID_2 values with null precipitation_MEAN:")
print(null_precipitation_ids)

In [ ]:
null_rows_era_chirps['ID_2'].unique()

In [ ]:
era_chirps_id_1993_2023

In [ ]:
null_rows_era_chirps['precipitation_MEAN'].isna()

Legacy code for discovering underrepresented ID_2 in climate variables 

In [7]:
def find_incomplete_ids(df, id_column='ID_2'):
    """
    Finds and returns IDs associated with rows that have any null values.

    Parameters:
    df (pd.DataFrame): The DataFrame to check.
    id_column (str): The column containing the ID values.

    Returns:
    list: IDs of rows with null values.
    """
    incomplete_ids = df[df.isnull().any(axis=1)][id_column].unique().tolist()

    print(f"Total rows with nulls: {df.isnull().any(axis=1).sum()}")
    print(f"Unique {id_column}s with nulls: {len(incomplete_ids)} -> {incomplete_ids}")

    return incomplete_ids

def fill_incomplete_polygons(dataframe, shapefile, incomplete_ids, stat_cols):
    """
    Fill rows with nulls by copying values from the nearest complete polygon.

    Parameters:
        dataframe (pd.DataFrame): Main DataFrame with 'ID_2' and data.
        shapefile (gpd.GeoDataFrame): GeoDataFrame with 'ID_2' and geometries.
        incomplete_ids (list): List of 'ID_2' values with incomplete data.
        stat_cols (list): Statistic columns to fill (optional).

    Returns:
        pd.DataFrame: Updated DataFrame with filled-in rows.
    """

    # Ensure shapefile uses a projected CRS
    shapefile = shapefile.to_crs(epsg=3857)
    shapefile["centroid"] = shapefile.geometry.centroid

    # Get valid (complete) polygons
    valid_ids = dataframe[~dataframe["ID_2"].isin(incomplete_ids)]["ID_2"].unique()
    non_missing_polygons = shapefile[shapefile["ID_2"].isin(valid_ids)].copy()

    # Map each incomplete ID to its nearest valid neighbor
    nearest_mapping = {}

    print("Finding nearest complete polygons for incomplete ID_2 values...")
    for ID in tqdm(incomplete_ids, desc="Computing nearest polygons"):
        centroid = shapefile[shapefile["ID_2"] == ID].centroid.iloc[0]
        non_missing_polygons["distance"] = non_missing_polygons["centroid"].distance(centroid)
        nearest_mapping[ID] = non_missing_polygons.loc[non_missing_polygons["distance"].idxmin(), "ID_2"]

    # Fill the incomplete rows
    updated_rows = []
    print("\nFilling incomplete data using nearest neighbors...")
    for ID in tqdm(incomplete_ids, desc="Filling rows"):
        original_rows = dataframe[dataframe["ID_2"] == ID]
        nearest_ID = nearest_mapping[ID]
        fill_source = dataframe[dataframe["ID_2"] == nearest_ID].iloc[0]

        for _, row in original_rows.iterrows():
            new_row = row.copy()
            for col in dataframe.columns:
                if pd.isnull(new_row[col]):
                    new_row[col] = fill_source[col]
            updated_rows.append(new_row)

    # Remove old incomplete rows and append updated ones
    clean_df = dataframe[~dataframe["ID_2"].isin(incomplete_ids)]
    updated_df = pd.concat([clean_df, pd.DataFrame(updated_rows)], ignore_index=True)

    print(f"\n✅ Incomplete data filled for {len(incomplete_ids)} ID_2 entries.")
    return updated_df


In [12]:
find_incomplete_ids(era_chirps_id_1993_2023_filled)

Total rows with nulls: 0
Unique ID_2s with nulls: 0 -> []


[]

In [14]:
era_chirps_id_1993_2023_filled.to_csv(INTERIM_DIR/ "era_chirps_id_1993_2023.csv", index=False)

In [ ]:
find_incomplete_ids(era_chirps_id_1993_2023, id_column='ID_2')

In [9]:
era_chirps_id_1993_2023_filled = fill_incomplete_polygons(dataframe=era_chirps_id_1993_2023,
    shapefile=in_shp, incomplete_ids=[123, 125, 153, 204, 240, 379], stat_cols=['temperature_2m_MEAN', 'temperature_2m_min_MEAN',
       'temperature_2m_max_MEAN', 'potential_evaporation_sum_MEAN',
       'total_evaporation_sum_MEAN', 'precipitation_MEAN'])

Finding nearest complete polygons for incomplete ID_2 values...


Computing nearest polygons: 100%|██████████| 6/6 [00:00<00:00, 1435.42it/s]



Filling incomplete data using nearest neighbors...


Filling rows: 100%|██████████| 6/6 [00:11<00:00,  1.93s/it]



✅ Incomplete data filled for 6 ID_2 entries.


In [10]:
era_chirps_id_1993_2023

,Unnamed: 0,Date,Doy,ID_2,temperature_2m_MEAN,temperature_2m_min_MEAN,temperature_2m_max_MEAN,potential_evaporation_sum_MEAN,total_evaporation_sum_MEAN,Region,precipitation_MEAN,YearMonth
0,0,1993-01-01,1,1,20.709194,16.977877,24.706811,-0.010296,-0.003956,Sumatra,2.037670,1993-01
1,1,1993-01-01,1,2,21.582465,18.095360,25.474377,-0.009332,-0.004078,Sumatra,0.262112,1993-01
2,2,1993-01-01,1,3,24.692348,22.235172,27.720153,-0.011643,-0.004529,Sumatra,0.000000,1993-01
3,3,1993-01-01,1,4,23.722732,20.910947,27.319734,-0.010711,-0.004267,Sumatra,0.748526,1993-01
4,4,1993-01-01,1,5,20.802229,17.112204,24.707693,-0.010373,-0.003945,Sumatra,3.548272,1993-01
...,...,...,...,...,...,...,...,...,...,...,...,...
5026963,5026963,2023-12-27,361,444,27.313731,24.367449,31.377478,-0.008531,-0.004548,Java,13.263003,2023-12
5026964,5026964,2023-12-28,362,444,26.902143,23.847837,31.970732,-0.008388,-0.004595,Java,13.246793,2023-12
5026965,5026965,2023-12-29,363,444,26.763019,23.781351,31.827548,-0.008483,-0.005088,Java,18.036463,2023-12
5026966,5026966,2023-12-30,364,444,26.403595,24.242135,30.929505,-0.006610,-0.004357,Java,12.833191,2023-12


Clean up era-chirps column name and convention.

In [ ]:
# First drop the 'Unnamed: 0' column from the era_chirps_id_1993_2023 DataFrame
era_chirps_id_1993_2023 = era_chirps_id_1993_2023.drop(columns=['Unnamed: 0'])

# Remove redundant suffixes from column names
columns_to_check = [
    'temperature_2m_MEAN', 'temperature_2m_min_MEAN',
    'temperature_2m_max_MEAN', 'potential_evaporation_sum_MEAN',
    'total_evaporation_sum_MEAN', 'precipitation_MEAN'
]
# Create a dictionary for renaming
rename_dict = {col: col.replace('_MEAN', '') for col in era_chirps_id_1993_2023.columns if col in columns_to_check and '_MEAN' in col}
era_chirps_id_1993_2023.rename(columns=rename_dict, inplace=True)
# Convert total evaporation and potential evaporation to positive values (ERA5 convention-->standard
# meteorological convention)
era_chirps_id_1993_2023['total_evaporation_sum'] = era_chirps_id_1993_2023['total_evaporation_sum'].abs()
era_chirps_id_1993_2023['potential_evaporation_sum'] = era_chirps_id_1993_2023['potential_evaporation_sum'].abs()

Create ID_2, daily statisitcs and evaporative stress index + aridity index. (NO LULC)

In [ ]:
# Create a copy of the era_chirps_id_1993_2023 DataFrame to avoid modifying the original
env_var = era_chirps_id_1993_2023.copy()
# Make sure the 'Date' column is in datetime format
env_var['Date'] = pd.to_datetime(env_var['Date'])
# Create a 'YearMonth' column for merging
env_var['YearMonth'] = env_var['Date'].dt.to_period('M')
# Convert YearMonth to timestamp for merging
env_var['YearMonth'] = env_var['YearMonth'].dt.to_timestamp()

# Create a matching 'YearMonth' column in enso dataframe
enso_sst['YearMonth'] = pd.to_datetime(enso_sst['Year'].astype(str) + '-' + enso_sst['Month'].astype(str).str.zfill(2))

# Columns to add from enso
enso_columns_to_add = ['YearMonth', 'NINO1+2', 'ANOM1+2', 'NINO3', 'ANOM3', 'NINO4', 'ANOM4', 'NINO3.4', 'ANOM3.4']

# Merge enso dataframes directly with env_var
merged_df = env_var.merge(enso_sst[enso_columns_to_add], on='YearMonth', how='left')

# First, melt dmi dataframe (convert from wide to long format)
dmi_long = dmi.melt(id_vars='Year', var_name='Month', value_name='DMI')
dmi_long['Month'] = dmi_long['Month'].astype(int)  # ensure Month is integer
dmi_long['YearMonth'] = pd.to_datetime(dmi_long['Year'].astype(str) + '-' + dmi_long['Month'].astype(str).str.zfill(2))

# Melt dmi_east dataframe
dmi_east_long = dmi_east.melt(id_vars='Year', var_name='Month', value_name='DMI_East')
dmi_east_long['Month'] = dmi_east_long['Month'].astype(int)
dmi_east_long['YearMonth'] = pd.to_datetime(dmi_east_long['Year'].astype(str) + '-' + dmi_east_long['Month'].astype(str).str.zfill(2))

# Merge DMI
merged_df = merged_df.merge(dmi_long[['YearMonth', 'DMI']], on='YearMonth', how='left')
# Merge DMI East
merged_df = merged_df.merge(dmi_east_long[['YearMonth', 'DMI_East']], on='YearMonth', how='left')

# Calculate evaporative stress index and aridity index
merged_df['evaporative_stress_index'] = merged_df['total_evaporation_sum']/merged_df['potential_evaporation_sum']
merged_df['aridity_index'] = merged_df['precipitation'] / merged_df['potential_evaporation_sum']


# Save full merged dataframe
merged_df.to_csv(IN_DIR / 'daily_env_id_1993_2023.csv', index=False)
# Filter the dataframe for the years 2010 to 2023
filtered_df = merged_df[(merged_df['YearMonth'].dt.year >= 2010) & (merged_df['YearMonth'].dt.year <= 2023)].reset_index(drop=True)
filtered_df.to_csv(IN_DIR / 'daily_env_id_2010_2023.csv', index=False)

Creating ID_2, monthly statistics. (+LULC)

In [ ]:
df = merged_df.copy()
# Ensure Date column is datetime
df['Date'] = pd.to_datetime(df['Date'])
# Drop 'Doy' and replace 'YearMonth' with first day of month
df['YearMonth'] = df['Date'].dt.to_period('M').dt.to_timestamp()

# Define relevant column sets
temperature_cols = ['temperature_2m', 'temperature_2m_min', 'temperature_2m_max']
evap_precip_cols = ['potential_evaporation_sum', 'total_evaporation_sum', 'precipitation']
monthly_cols = ['NINO1+2', 'ANOM1+2', 'NINO3', 'ANOM3', 'NINO4', 'ANOM4', 'NINO3.4',
                'ANOM3.4', 'DMI', 'DMI_East']
keep_cols = ['YearMonth', 'ID_2', 'Region'] + temperature_cols + evap_precip_cols + monthly_cols

# Filter necessary columns
df = df[keep_cols]

# Group by YearMonth and ID_2
monthly_df = df.groupby(['YearMonth', 'ID_2'], as_index=False).agg({
    **{col: 'mean' for col in temperature_cols},
    **{col: 'sum' for col in evap_precip_cols},
    **{col: 'first' for col in monthly_cols},  # Since these are already monthly
    'Region': 'first'  # Assuming region does not change within ID_2
})

# Recompute evaporative_stress_index and aridity_index
monthly_df['evaporative_stress_index'] = (
    monthly_df['total_evaporation_sum'] / monthly_df['potential_evaporation_sum'])
monthly_df['aridity_index'] = (
    monthly_df['precipitation'] / monthly_df['potential_evaporation_sum'])

# Add lulc class columns
# 1. Create a mapping of ID_2 to Region from dengue_by_ID_2
id2_region_map = era_chirps_id_1993_2023[['ID_2', 'Region']].drop_duplicates()
# 2. Add 'Region' column to lulc_final
lulc_200_full = pd.merge(lulc_200_full, id2_region_map, on='ID_2', how='left')

# Define LULC class columns
class_columns = [
    'Class_70', 'Class_60', 'Class_50', 'Class_40', 'Class_95',
    'Class_30', 'Class_20', 'Class_10', 'Class_90', 'Class_80'
]

# Compute True_Area_* columns based on class proportions and Class_sum
for col in class_columns:
    lulc_200_full[f'True_Area_{col}'] = lulc_200_full[col] * lulc_200_full['Class_sum']

# Select only the necessary columns from lulc_200_full
true_area_cols = [f'True_Area_{col}' for col in class_columns]
all_lulc_columns = ['ID_2', 'Class_sum'] + class_columns + true_area_cols

# Merge into monthly_df
monthly_df = pd.merge(monthly_df, lulc_200_full[all_lulc_columns], on='ID_2', how='left')
monthly_df.to_csv(IN_DIR / 'monthly_env_id_1993_2023.csv', index=False)

filtered_monthly_df = monthly_df[(monthly_df['YearMonth'].dt.year >= 2010) & 
                                  (monthly_df['YearMonth'].dt.year <= 2023)].reset_index(drop=True)
filtered_monthly_df.to_csv(IN_DIR / 'monthly_env_id_2010_2023.csv', index=False)

Turning daily statistics to monthly statistics -- BY REGION

In [ ]:
# Assuming your dataframe is named 'env_var'
env_var = era_chirps_id_1993_2023.copy()
# Make sure the 'Date' column is in datetime format
env_var['Date'] = pd.to_datetime(env_var['Date'])
# Create a 'YearMonth' column for grouping
env_var['YearMonth'] = env_var['Date'].dt.to_period('M')

# Group by 'Region' and 'YearMonth', then calculate the mean for relevant columns
monthly_env_mean = env_var.groupby(['Region', 'YearMonth'])[
    ['temperature_2m', 'temperature_2m_min', 'temperature_2m_max']
].mean().reset_index()

# Group by 'Region' and 'YearMonth', then calculate the sum for relevant columns
monthly_env_sum = env_var.groupby(['Region', 'YearMonth'])[
    ['precipitation','potential_evaporation_sum', 'total_evaporation_sum']
].sum().reset_index()

# Merge the mean and sum dataframes
monthly_env_data = pd.merge(monthly_env_mean, monthly_env_sum, on=['Region', 'YearMonth'], how='inner')

# If YearMonth is a Period (like Period('2023-01', 'M')), convert it to timestamp
monthly_env_data['YearMonth'] = monthly_env_data['YearMonth'].dt.to_timestamp()

# Create a matching 'YearMonth' column in enso dataframe
enso_sst['YearMonth'] = pd.to_datetime(enso_sst['Year'].astype(str) + '-' + enso_sst['Month'].astype(str).str.zfill(2))

# Columns to add from enso
enso_columns_to_add = ['YearMonth', 'NINO1+2', 'ANOM1+2', 'NINO3', 'ANOM3', 'NINO4', 'ANOM4', 'NINO3.4', 'ANOM3.4']

# Merge enso dataframes
merged_df = monthly_env_data.merge(enso_sst[enso_columns_to_add], on='YearMonth', how='left')

# First, melt dmi dataframe (convert from wide to long format)
dmi_long = dmi.melt(id_vars='Year', var_name='Month', value_name='DMI')
dmi_long['Month'] = dmi_long['Month'].astype(int)  # ensure Month is integer
dmi_long['YearMonth'] = pd.to_datetime(dmi_long['Year'].astype(str) + '-' + dmi_long['Month'].astype(str).str.zfill(2))

# Melt dmi_east dataframe
dmi_east_long = dmi_east.melt(id_vars='Year', var_name='Month', value_name='DMI_East')
dmi_east_long['Month'] = dmi_east_long['Month'].astype(int)
dmi_east_long['YearMonth'] = pd.to_datetime(dmi_east_long['Year'].astype(str) + '-' + dmi_east_long['Month'].astype(str).str.zfill(2))

# Merge DMI
merged_df = merged_df.merge(dmi_long[['YearMonth', 'DMI']], on='YearMonth', how='left')
# Merge DMI East
merged_df = merged_df.merge(dmi_east_long[['YearMonth', 'DMI_East']], on='YearMonth', how='left')


# 1. Create a mapping of ID_2 to Region from dengue_by_ID_2
id2_region_map = era_chirps_id_1993_2023[['ID_2', 'Region']].drop_duplicates()
# 2. Add 'Region' column to lulc_final
lulc_200_full = pd.merge(lulc_200_full, id2_region_map, on='ID_2', how='left')

# Define the list of columns to aggregate
area_columns = [
    'True_Area_Class_70', 'True_Area_Class_60', 'True_Area_Class_50',
    'True_Area_Class_40', 'True_Area_Class_95', 'True_Area_Class_30',
    'True_Area_Class_20', 'True_Area_Class_10', 'True_Area_Class_90',
    'True_Area_Class_80', 'Class_sum'
]
class_columns = ['Class_70', 'Class_60', 'Class_50', 'Class_40', 'Class_95', 
                 'Class_30', 'Class_20', 'Class_10', 'Class_90', 'Class_80']
for col in class_columns:
    lulc_200_full[f'True_Area_{col}'] = lulc_200_full[col] * lulc_200_full['Class_sum']

# 3. Aggregate lulc_final_with_region by 'Region'
# We sum the area columns to get total area per region for each class
lulc_agg_by_region = lulc_200_full.groupby('Region')[area_columns].sum().reset_index()

# 4. Merge the aggregated LULC data into monthly_dengue_env_region
# Since LULC data is static (not time-series in this context), it's merged only on 'Region'
merged_df = pd.merge(
    merged_df,
    lulc_agg_by_region,
    on='Region',
    how='inner' # Use 'inner' to ensure only matching regions are kept
)

true_area_class_columns = [
    'True_Area_Class_70', 'True_Area_Class_60', 'True_Area_Class_50',
    'True_Area_Class_40', 'True_Area_Class_95', 'True_Area_Class_30',
    'True_Area_Class_20', 'True_Area_Class_10', 'True_Area_Class_90',
    'True_Area_Class_80']
# Create new percentage columns
for col in true_area_class_columns:
    # Extract the class number from the column name (e.g., '70' from 'True_Area_Class_70')
    class_num = col.replace('True_Area_Class_', '')
    new_col_name = f'Class_{class_num}'
    # Calculate the percentage. Handle potential division by zero by checking if 'Class_sum' is 0
    # If Class_sum is 0, the percentage for that row will be 0.
    merged_df[new_col_name] = merged_df.apply(
        lambda row: row[col] / row['Class_sum'] if row['Class_sum'] != 0 else 0,
        axis=1
    )

# Recompute evaporative_stress_index and aridity_index
merged_df['evaporative_stress_index'] = (
    merged_df['total_evaporation_sum'] / merged_df['potential_evaporation_sum'])
merged_df['aridity_index'] = (
    merged_df['precipitation'] / merged_df['potential_evaporation_sum'])
# Save the final merged dataframe
merged_df.to_csv(IN_DIR / 'monthly_env_region_1993_2023.csv', index=False)

# Filter the dataframe
filtered_df = merged_df[(merged_df['YearMonth'].dt.year >= 2010) & (merged_df['YearMonth'].dt.year <= 2023)].reset_index(drop=True)
filtered_df.to_csv(IN_DIR / 'monthly_env_region_2010_2023.csv', index=False)

National Level

In [ ]:
# Get all True_Area columns for national summary of lulc percentages
true_area_cols = [col for col in lulc_200_full.columns if col.startswith("True_Area_Class_")]
# Sum true area for each class
true_area_totals = lulc_200_full[true_area_cols].sum()
# Total area across all classes
total_area = true_area_totals.sum()
# Build proportion columns named Class_{i}
class_proportions = {
    f"Class_{col.split('_')[-1]}": (true_area_totals[col] / total_area).round(6)
    for col in true_area_cols
}
# Combine all into one row
summary_row = {
    **class_proportions,
    **true_area_totals.to_dict(),
    "Class_sum": total_area
}
# Create DataFrame
lulc_200_national = pd.DataFrame([summary_row])

In [ ]:
# Assuming your dataframe is named 'env_var'
env_var = era_chirps_id_1993_2023.copy()
# Make sure the 'Date' column is in datetime format
env_var['Date'] = pd.to_datetime(env_var['Date'])

# Create a 'YearMonth' column for grouping
env_var['YearMonth'] = env_var['Date'].dt.to_period('M')

# Group by 'Region' and 'YearMonth', then calculate the mean for relevant columns
monthly_env_mean = env_var.groupby(['YearMonth'])[
    ['temperature_2m', 'temperature_2m_min', 'temperature_2m_max']
].mean().reset_index()

# Group by 'Region' and 'YearMonth', then calculate the sum for relevant columns
monthly_env_sum = env_var.groupby(['YearMonth'])[
    ['precipitation','potential_evaporation_sum', 'total_evaporation_sum']
].sum().reset_index()

# Merge the mean and sum dataframes
monthly_env_data = pd.merge(monthly_env_mean, monthly_env_sum, on=['YearMonth'], how='inner')

# If YearMonth is a Period (like Period('2023-01', 'M')), convert it to timestamp
monthly_env_data['YearMonth'] = monthly_env_data['YearMonth'].dt.to_timestamp()

# Create a matching 'YearMonth' column in enso dataframe
enso_sst['YearMonth'] = pd.to_datetime(enso_sst['Year'].astype(str) + '-' + enso_sst['Month'].astype(str).str.zfill(2))

# Columns to add from enso
enso_columns_to_add = ['YearMonth', 'NINO1+2', 'ANOM1+2', 'NINO3', 'ANOM3', 'NINO4', 'ANOM4', 'NINO3.4', 'ANOM3.4']

# Merge enso dataframes
merged_df = monthly_env_data.merge(enso_sst[enso_columns_to_add], on='YearMonth', how='left')

# First, melt dmi dataframe (convert from wide to long format)
dmi_long = dmi.melt(id_vars='Year', var_name='Month', value_name='DMI')
dmi_long['Month'] = dmi_long['Month'].astype(int)  # ensure Month is integer
dmi_long['YearMonth'] = pd.to_datetime(dmi_long['Year'].astype(str) + '-' + dmi_long['Month'].astype(str).str.zfill(2))

# Melt dmi_east dataframe
dmi_east_long = dmi_east.melt(id_vars='Year', var_name='Month', value_name='DMI_East')
dmi_east_long['Month'] = dmi_east_long['Month'].astype(int)
dmi_east_long['YearMonth'] = pd.to_datetime(dmi_east_long['Year'].astype(str) + '-' + dmi_east_long['Month'].astype(str).str.zfill(2))

# Merge DMI
merged_df = merged_df.merge(dmi_long[['YearMonth', 'DMI']], on='YearMonth', how='left')
# Merge DMI East
merged_df = merged_df.merge(dmi_east_long[['YearMonth', 'DMI_East']], on='YearMonth', how='left')

# Recompute evaporative_stress_index and aridity_index
merged_df['evaporative_stress_index'] = (
    merged_df['total_evaporation_sum'] / merged_df['potential_evaporation_sum'])
merged_df['aridity_index'] = (
    merged_df['precipitation'] / merged_df['potential_evaporation_sum'])

# Repeat summary_df to match the number of rows in merged_df
lulc_replicated = pd.concat([lulc_200_national] * len(merged_df), ignore_index=True)
# Concatenate along columns
merged_df = pd.concat([merged_df.reset_index(drop=True), lulc_replicated], axis=1)

merged_df.to_csv(IN_DIR / 'monthly_env_national_1993_2023.csv', index=False)

# Filter the dataframe
filtered_df = merged_df[(merged_df['YearMonth'].dt.year >= 2010) & (merged_df['YearMonth'].dt.year <= 2023)].reset_index(drop=True)
filtered_df.to_csv(IN_DIR / 'monthly_env_national_2010_2023.csv', index=False)

In [ ]:
filtered_df

In [ ]:
era_df = era_1993_2023.merge(in_shp[['ID_2', 'Region']], on='ID_2')
era_df['Date'] = pd.to_datetime(era_df['Date'])
era_df['Month'] = era_df['Date'].dt.month

climatology_vars = ['temperature_2m_MEAN', 'temperature_2m_min_MEAN',
       'temperature_2m_max_MEAN', 'potential_evaporation_sum_MEAN']
era_climatology = era_df.groupby(['Region', 'Month'])[climatology_vars].mean().reset_index()
era_merged = era_df.merge(era_climatology, on=['Region', 'Month'], suffixes=('', '_clim'))
for var in climatology_vars:
    era_merged[f'{var}_anomaly'] = era_merged[var] - era_merged[f'{var}_clim']
era_anomaly = era_merged.groupby(['Region', 'Date'])[
    [f'{var}_anomaly' for var in climatology_vars]
].mean().reset_index()

regions = era_anomaly['Region'].unique()
for var in climatology_vars:
    for region in regions:
        subset = era_anomaly[era_anomaly['Region'] == region]
        
        plt.figure(figsize=(12, 5))
        sns.lineplot(data=subset, x='Date', y=f'{var}_anomaly')
        plt.title(f"{var.capitalize()} Anomaly in {region} (1993–2023)")
        plt.ylabel(f"{var.capitalize()} Anomaly")
        plt.grid(True)
        plt.tight_layout()

        # Save to FIGURE_DIR
        filename = f"{var}_anomaly_{region.replace(' ', '_')}.png"
        plt.savefig(os.path.join(FIGURE_DIR, filename), dpi=300)
        plt.close()  # Close to avoid memory issues when looping

In [ ]:
era_df = era_1993_2023.merge(in_shp[['ID_2', 'Region']], on='ID_2')
era_df['Date'] = pd.to_datetime(era_df['Date'])
era_df['Month'] = era_df['Date'].dt.month

climatology_vars = ['temperature_2m_MEAN', 'temperature_2m_min_MEAN',
       'temperature_2m_max_MEAN', 'potential_evaporation_sum_MEAN']
era_climatology = era_df.groupby(['Region', 'Month'])[climatology_vars].mean().reset_index()
era_merged = era_df.merge(era_climatology, on=['Region', 'Month'], suffixes=('', '_clim'))

for var in climatology_vars:
    era_merged[f'{var}_anomaly'] = era_merged[var] - era_merged[f'{var}_clim']
era_anomaly = era_merged.groupby(['Region', 'Date'])[
    [f'{var}_anomaly' for var in climatology_vars]
].mean().reset_index()
for var in climatology_vars:
    plt.figure(figsize=(14, 6))
    sns.lineplot(data=era_anomaly, x='Date', y=f'{var}_anomaly', hue='Region')
    plt.title(f"ERA {var.capitalize()} Anomaly (1993–2023) All Regions")
    plt.ylabel(f"{var.capitalize()} Anomaly")
    plt.grid(True)
    plt.tight_layout()
    # Save to FIGURE_DIR
    filename = f"{var}_anomaly_all_regions.png"
    plt.savefig(os.path.join(FIGURE_DIR, filename), dpi=300)
    plt.close()


In [ ]:
chirsp_df = chirsp_1993_2023.merge(in_shp[['ID_2', 'Region']], on='ID_2')
chirsp_df['Date'] = pd.to_datetime(chirsp_df['Date'])
chirsp_df['Month'] = chirsp_df['Date'].dt.month

climatology_vars = ['precipitation_MEAN']
chirsp_climatology = chirsp_df.groupby(['Region', 'Month'])[climatology_vars].mean().reset_index()
chirsp_merged = chirsp_df.merge(chirsp_climatology, on=['Region', 'Month'], suffixes=('', '_clim'))
for var in climatology_vars:
    chirsp_merged[f'{var}_anomaly'] = chirsp_merged[var] - chirsp_merged[f'{var}_clim']
chirsp_anamoly = chirsp_merged.groupby(['Region', 'Date'])[
    [f'{var}_anomaly' for var in climatology_vars]
].mean().reset_index()

regions = chirsp_anamoly['Region'].unique()
for var in climatology_vars:
    for region in regions:
        subset = chirsp_anamoly[chirsp_anamoly['Region'] == region]
        
        plt.figure(figsize=(12, 5))
        sns.lineplot(data=subset, x='Date', y=f'{var}_anomaly')
        plt.title(f"{var.capitalize()} Anomaly in {region} (1993–2023)")
        plt.ylabel(f"{var.capitalize()} Anomaly")
        plt.grid(True)
        plt.tight_layout()

        # Save to FIGURE_DIR
        filename = f"{var}_anomaly_{region.replace(' ', '_')}.png"
        plt.savefig(os.path.join(FIGURE_DIR, filename), dpi=300)
        plt.close()  # Close to avoid memory issues when looping

In [ ]:
import matplotlib.pyplot as plt

# Make sure the datetime column is in datetime format
updated_era_1993_2023['Date'] = pd.to_datetime(updated_era_1993_2023['Date'])

# Count occurrences of each ID_2
id_counts = updated_era_1993_2023['ID_2'].value_counts().sort_index()

# Get the expected (most common) count
expected_count = id_counts.mode()[0]

# Get IDs that occur less than expected
underrepresented_ids = id_counts[id_counts < expected_count].index

# Filter the dataframe to just those IDs
filtered_df = updated_era_1993_2023[updated_era_1993_2023['ID_2'].isin(underrepresented_ids)]

# Plot
plt.figure(figsize=(15, 6))
plt.scatter(filtered_df['Date'], filtered_df['ID_2'], alpha=0.6, s=10, color='red')
plt.title('Occurrences of Underrepresented ID_2 Values Over Time')
plt.xlabel('Date')
plt.ylabel('ID_2')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Make sure the datetime column is in datetime format
era_1993_2023['Date'] = pd.to_datetime(era_1993_2023['Date'])

# Count occurrences of each ID_2
id_counts = era_1993_2023['ID_2'].value_counts().sort_index()

# Get the expected (most common) count
expected_count = id_counts.mode()[0]

# Get IDs that occur less than expected
underrepresented_ids = id_counts[id_counts < expected_count].index

# Filter the dataframe to just those IDs
filtered_df = era_1993_2023[era_1993_2023['ID_2'].isin(underrepresented_ids)]

# Plot
plt.figure(figsize=(15, 6))
plt.scatter(filtered_df['Date'], filtered_df['ID_2'], alpha=0.6, s=10, color='red')
plt.title('Occurrences of Underrepresented ID_2 Values Over Time')
plt.xlabel('Date')
plt.ylabel('ID_2')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# Count the number of occurrences of each ID_2
id_counts = prec['ID_2'].value_counts().sort_index()

# Determine the expected count (most common count)
expected_count = id_counts.mode()[0]

# Find ID_2s with counts less than the expected
underrepresented_ids = id_counts[id_counts < expected_count]

# Display the result
print("Expected count per ID_2:", expected_count)
print("IDs with less than expected count:")
print(underrepresented_ids)


In [ ]:
import pandas as pd

# Count the number of occurrences of each ID_2
id_counts = updated_era_1993_2023['ID_2'].value_counts().sort_index()

# Determine the expected count (most common count)
expected_count = id_counts.mode()[0]

# Find ID_2s with counts less than the expected
underrepresented_ids = id_counts[id_counts < expected_count]

# Display the result
print("Expected count per ID_2:", expected_count)
print("IDs with less than expected count:")
print(underrepresented_ids)


In [ ]:
era_2010_2023 = era_2010_2023.drop(columns=['total_evaporation_sum_MEAN','total_precipitation_sum_MEAN'])

In [ ]:
import pandas as pd

# Merging era_1993_1999 with era_2005_2009, selecting only relevant columns
merged_df1 = pd.merge(era_1993_1999, era_2005_2009[['Date', 'ID_2', 'Doy', 'potential_evaporation_sum_MEAN']],
                      on=['Date', 'ID_2', 'Doy'], how='outer')

# Merging the result with era5_temp_2000_2009, selecting only relevant columns
final_df = pd.merge(merged_df1, era5_temp_2000_2009[['Date', 'ID_2', 'Doy', 'temperature_2m_MEAN', 
                                                     'temperature_2m_min_MEAN', 'temperature_2m_max_MEAN']],
                     on=['Date', 'ID_2', 'Doy'], how='outer')

# Fill missing values from columns with the other column values where necessary
final_df['temperature_2m_MEAN'] = final_df['temperature_2m_MEAN_x'].fillna(final_df['temperature_2m_MEAN_y'])
final_df['temperature_2m_min_MEAN'] = final_df['temperature_2m_min_MEAN_x'].fillna(final_df['temperature_2m_min_MEAN_y'])
final_df['temperature_2m_max_MEAN'] = final_df['temperature_2m_max_MEAN_x'].fillna(final_df['temperature_2m_max_MEAN_y'])
final_df['potential_evaporation_sum_MEAN'] = final_df['potential_evaporation_sum_MEAN_x'].fillna(final_df['potential_evaporation_sum_MEAN_y'])

# Drop the old '_x' and '_y' columns
final_df = final_df.drop(columns=['temperature_2m_MEAN_x', 'temperature_2m_MEAN_y', 
                                  'temperature_2m_min_MEAN_x', 'temperature_2m_min_MEAN_y',
                                  'temperature_2m_max_MEAN_x', 'temperature_2m_max_MEAN_y',
                                  'potential_evaporation_sum_MEAN_x', 'potential_evaporation_sum_MEAN_y'])

# Display the final dataframe
print(final_df)


In [ ]:
era_1993_2023 = pd.concat([final_df,era_2010_2023])

In [ ]:
# Step 1: Group by 'Date' and count unique 'ID_2' for each 'Date'
unique_id_count_per_date = era_1993_2023.groupby('Date')['ID_2'].nunique()

# Step 2: Check if the count of unique 'ID_2' for each 'Date' is 444
incorrect_dates = unique_id_count_per_date[unique_id_count_per_date != 444]

# Step 3: Output the result
incorrect_dates


Visualize null values for precipitation.

In [ ]:
# Define the column to check for null values
column_to_check = 'precipitation_MEAN'

# Create an empty dictionary to store plots for each year
for year, group in prec.groupby('Year'):
    plt.figure(figsize=(10, 6))
    
    # Count the number of null values for each day in the column
    null_count = group[column_to_check].isnull().groupby(group['Date']).sum()
    
    # Plot the null count
    plt.plot(null_count.index, null_count, label=column_to_check, color='blue')
    
    # Add title and labels
    plt.title(f"Null Value Count (Precipitation) for Each Day in {year}")
    plt.xlabel('Date')
    plt.ylabel('Null Value Count')
    plt.legend()

    # Save the plot for the year
    save_path = f"{save_dir}/nulls_per_day_{year}_precipitation.png"
    plt.savefig(save_path)
    plt.close()  # Close the plot to avoid displaying it inline


In [ ]:
# Now check by ID, creating one bar plot for each year
# Iterate over each year and create the plot
for year, group in prec.groupby('Year'):
    # Initialize a plot for the current year
    plt.figure(figsize=(12, 6))
    
    # Initialize a list to store the null counts
    null_counts_precipitation = []
    id_2_values = []

    # Iterate over each unique 'ID_2' and calculate the null counts
    for id_2, sub_group in group.groupby('ID_2'):
        # Count null values for 'precipitation_MEAN' for the current 'ID_2'
        null_count_precipitation = sub_group['precipitation_MEAN'].isnull().sum()

        # Store the null counts and 'ID_2'
        null_counts_precipitation.append(null_count_precipitation)
        id_2_values.append(id_2)

    # Create the bar plot
    width = 0.35  # Bar width
    x = range(len(id_2_values))  # X positions for bars

    # Bar plot for 'precipitation_MEAN'
    plt.bar(x, null_counts_precipitation, width, label='precipitation_MEAN Nulls', color='blue')

    # Add labels and title
    plt.xlabel('ID_2')
    plt.ylabel('Null Value Count')
    plt.title(f'Null Value Counts (Prec) for Each ID_2 in {year}')
    plt.xticks([p + width / 2 for p in x], id_2_values, rotation=45)
    plt.legend()

    # Save the plot for the current year
    save_path = f"{save_dir}/nulls_per_id_{year}_prec.png"
    plt.tight_layout()  # Adjust layout to avoid overlap
    plt.savefig(save_path)
    plt.close()  # Close the plot to avoid displaying it inline

Visualise null values for NDVI.

In [ ]:
# Define columns (NDVI_MEAN and EVI_MEAN)
columns_to_check = ['NDVI_MEAN', 'EVI_MEAN']
colors = ['red', 'blue']
labels = ['NDVI_MEAN', 'EVI_MEAN']

# Iterate over each year to create plots of time
for year, group in ndvi_all.groupby('Year'):
    plt.figure(figsize=(10, 6))

    null_percentages = {}  # Store null percentages for each column
    total_counts = group['Date'].value_counts().sort_index()  # Total entries per day
    
    for column, color, label in zip(columns_to_check, colors, labels):
        # Calculate null counts and convert to percentage
        null_counts = group[column].isnull().groupby(group['Date']).sum()
        null_percentages[column] = (null_counts / total_counts * 100).sort_index()
        
        # Plot each percentage curve
        plt.plot(null_percentages[column].index, null_percentages[column], label=label, color=color)

    # Highlight overlapping regions where both columns have the same percentage of missing data
    if 'NDVI_MEAN' in null_percentages and 'EVI_MEAN' in null_percentages:
        overlap = null_percentages['NDVI_MEAN'] == null_percentages['EVI_MEAN']
        plt.fill_between(null_percentages['NDVI_MEAN'].index, null_percentages['NDVI_MEAN'], 
                         where=overlap, color='gray', alpha=0.3, label="Overlap")

    # Update plot title, labels, and legend
    plt.title(f"Missing Data Percentage (NDVI & EVI) for Each Day in {year}")
    plt.xlabel('Date')
    plt.ylabel('Missing Data Percentage (%)')
    plt.legend()

    # Force y-axis to always go from 0 to 100%
    plt.ylim(0, 100)

    # Save the plot
    save_path = f"{save_dir}/nulls_percentage_per_day_{year}_ndvi.png"
    plt.savefig(save_path)
    plt.close()


In [ ]:
# Now check by ID, creating one bar plot for each year
# Iterate over each year and create the plot
for year, group in ndvi_all.groupby('Year'):
    # Initialize a plot for the current year
    plt.figure(figsize=(12, 6))

    # Initialize lists to store null counts and ID_2 values
    null_counts = []
    id_2_values = []

    # Iterate over each unique 'ID_2' and calculate the null counts
    for id_2, sub_group in group.groupby('ID_2'):
        # Count null values for 'NDVI_MEAN' (since EVI_MEAN is redundant)
        null_count = sub_group['EVI_MEAN'].isnull().sum()

        # Store the null count and 'ID_2'
        null_counts.append(null_count)
        id_2_values.append(id_2)

    # Create the bar plot
    plt.bar(id_2_values, null_counts, color='red', label='Null Counts (NDVI & EVI)')

    # Add labels and title
    plt.xlabel('ID_2')
    plt.ylabel('Null Value Count')
    plt.title(f'Null Value Counts for Each ID_2 in {year}')
    plt.xticks(rotation=45)
    plt.legend()

    # Save the plot for the current year
    save_path = f"{save_dir}/nulls_per_id_{year}_ndvi_evi.png"
    plt.tight_layout()  # Adjust layout to avoid overlap
    plt.savefig(save_path)
    plt.close()  # Close the plot to avoid displaying it inline

print('Barplot complete, generating choropleth maps.')


# Because there are a fair few IDs with missing value, create choropleth to better visualise spatiality
# Iterate over each year and create the choropleth map
for year, group in ndvi_all.groupby('Year'):
    null_counts_df = group.groupby('ID_2')['NDVI_MEAN'].apply(lambda x: x.isnull().sum()).reset_index()
    
    # Merge with shapefile
    merged = in_shp.merge(null_counts_df, on='ID_2', how='left')
    
    # Initialize a plot for the current year
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    merged.plot(column='NDVI_MEAN', cmap='OrRd', linewidth=0.8, edgecolor='black', legend=True, ax=ax)
    
    # Add labels and title
    plt.title(f'Choropleth Map of Null Counts for NDVI_MEAN in {year}')
    plt.axis('off')
    
    # Save the plot for the current year
    save_path = f"{save_dir}/choropleth_nulls_{year}_NDVI_MEAN.png"
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

print('Choropleth maps complete.')

In [ ]:
# Define columns for LST analysis
columns_to_check = ['LST_Day_MEAN', 'LST_Night_MEAN', 'LST_Mean_MEAN']
colors = ['red', 'blue', 'green']
labels = ['LST Day', 'LST Night', 'LST Mean']

# Iterate over each year to create plots of time
for year, group in lst.groupby('Year'):
    plt.figure(figsize=(10, 6))

    null_percentages = {}  # Store null percentages for each column
    total_counts = group['Date'].value_counts().sort_index()  # Total entries per day
    
    for column, color, label in zip(columns_to_check, colors, labels):
        # Calculate null counts and convert to percentage
        null_counts = group[column].isnull().groupby(group['Date']).sum()
        null_percentages[column] = (null_counts / total_counts * 100).sort_index()
        
        # Plot each percentage curve
        plt.plot(null_percentages[column].index, null_percentages[column], label=label, color=color)

    # Highlight overlapping regions where all columns have the same percentage of missing data
    if len(columns_to_check) == 3:
        overlap = (null_percentages['LST_Day_MEAN'] == null_percentages['LST_Night_MEAN']) & \
                  (null_percentages['LST_Day_MEAN'] == null_percentages['LST_Mean_MEAN'])
        plt.fill_between(null_percentages['LST_Day_MEAN'].index, null_percentages['LST_Day_MEAN'], 
                         where=overlap, color='gray', alpha=0.3, label="Full Overlap")

    # Update plot title, labels, and legend
    plt.title(f"Missing Data Percentage (LST) for Each Day in {year}")
    plt.xlabel('Date')
    plt.ylabel('Missing Data Percentage (%)')
    plt.legend()

    # Force y-axis to always go from 0 to 100%
    plt.ylim(0, 100)

    # Save the plot
    save_path = f"{save_dir}/nulls_percentage_per_day_{year}_lst.png"
    plt.savefig(save_path)
    plt.close()


In [ ]:
# Define the columns of interest
columns_to_check = ['LST_Day_MEAN', 'LST_Night_MEAN', 'LST_Mean_MEAN']

# Iterate over each year and create the plot for each column
for year, group in lst.groupby('Year'):
    for col in columns_to_check:
        # Initialize a plot for the current year and column
        plt.figure(figsize=(12, 6))
        
        # Initialize lists to store the null counts for each ID_2
        null_counts = []
        id_2_values = []

        # Iterate over each unique 'ID_2' and calculate the null counts
        for id_2, sub_group in group.groupby('ID_2'):
            null_counts.append(sub_group[col].isnull().sum())
            id_2_values.append(id_2)

        # Create the bar plot
        plt.bar(id_2_values, null_counts, color='blue')
        
        # Add labels and title
        plt.xlabel('ID_2')
        plt.ylabel('Null Value Count')
        plt.title(f'Null Value Counts for {col} in {year}')
        plt.xticks(rotation=45)
        
        # Save the plot for the current year and column
        save_path = f"{save_dir}/nulls_per_id_{year}_{col}.png"
        plt.tight_layout()  # Adjust layout to avoid overlap
        plt.savefig(save_path)
        plt.close()  # Close the plot to avoid displaying it inline


In [ ]:
lulc_100['ID_2'].nunique()

In [ ]:
classes = ['Class_10', 'Class_20', 'Class_30', 'Class_40', 'Class_50', 
           'Class_60', 'Class_70', 'Class_80', 'Class_90', 'Class_95']

# Plot setup
ax = lulc_200.set_index('ID_2')[classes].plot(
    kind='bar',
    stacked=True,
    figsize=(24, 6),
    colormap='tab10',
    edgecolor='none',
    width=1  )

# Add labels and title
plt.ylabel('Percentage of Area')
plt.xlabel('Region (ID_2)')
plt.title('Land Use/Land Cover Composition by Region')
plt.legend(title='Class', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(
    ticks=range(0, len(lulc_200['ID_2']), 5),  # Set ticks every 5 entries
    labels=lulc_200['ID_2'][::5],  # Show only every 5th label
    rotation=45
)
plt.tight_layout()
save_path = f"{save_dir}/lulc_200.png"
plt.savefig(save_path, dpi=600)
plt.close()  # Close the plot to avoid displaying it inline

In [ ]:
in_shp['NAME_1'].unique()

In [ ]:
in_shp['Region'] 

In [ ]:
sumatra = ['Aceh', 'Bangka-Belitung', 'Bengkulu', 'Jambi', 'Kepulauan Riau', 'Lampung', 'Sumatera Barat', 'Sumatera Selatan', 'Sumatera Utara' ]
nusa_tenggara = ['Bali', 'Nusa Tenggara Barat', 'Nusa Tenggara Timur']
java = ['Banten', 'Jawa Barat', 'Jawa Tengah', 'Jawa Timur', 'Riau', 'Yogyakarta', 'Jakarta Raya']
sulawesi = ['Gorontalo','Bengkulu', 'Sulawesi Barat', 'Sulawesi Selatan', 'Sulawesi Tengah', 'Sulawesi Tenggara', 'Sulawesi Utara']
kalimantan = ['Kalimantan Barat', 'Kalimantan Selatan', 'Kalimantan Tengah', 'Kalimantan Timur', 'Kalimantan Utara']
maluku_islands = ['Maluku Utara', 'Maluku']
papua = ['Papua', 'Irian Jaya Barat']

# Create a region lookup dictionary
region_mapping = {province: 'Sumatra' for province in sumatra}
region_mapping.update({province: 'Nusa Tenggara' for province in nusa_tenggara})
region_mapping.update({province: 'Java' for province in java})
region_mapping.update({province: 'Sulawesi' for province in sulawesi})
region_mapping.update({province: 'Kalimantan' for province in kalimantan})
region_mapping.update({province: 'Maluku Islands' for province in maluku_islands})
region_mapping.update({province: 'Papua' for province in papua})

# Assign regions to a new column based on NAME_1
in_shp['Region'] = in_shp['NAME_1'].map(region_mapping).fillna('Unknown')

In [ ]:
in_shp.to_file(r'D:\Projects\TMU\gee_dengue\Dengue_IN\in_shp\in_shp_islands')

In [ ]:
'''irian jaya barat => papua barat
Jakarta Raya => Jakarta Special Capital Region'''


In [ ]:
in_shp

In [ ]:
# Define columns for LST analysis
columns_to_check = ['LST_Day_MEAN', 'LST_Night_MEAN', 'LST_Mean_MEAN']
colors = ['red', 'blue', 'green']
labels = ['LST Day', 'LST Night', 'LST Mean']

plt.figure(figsize=(14, 8))
null_percentages = {col: [] for col in columns_to_check}
all_dates = []

# Process each year and collect data
for year, group in lst.groupby('Year'):
    # Ensure Date is a proper datetime format
    group['Date'] = pd.to_datetime(group['Date'], errors='coerce')
    group = group.dropna(subset=['Date'])  # Drop rows with invalid dates
    total_counts = group['Date'].value_counts().sort_index()
    all_dates.extend(group['Date'].sort_values().unique())

    for column in columns_to_check:
        # Calculate null counts and convert to percentage
        null_counts = group[column].isnull().groupby(group['Date']).sum()
        percentages = (null_counts / total_counts * 100).sort_index()
        null_percentages[column].extend(percentages)
    
    # Add a red line to separate years
    plt.axvline(x=len(all_dates) - 1, color='red', linestyle='--', linewidth=1, label=f'Separator {year}')

# Ensure dates are sorted correctly
all_dates = sorted(pd.to_datetime(all_dates))

# Apply smoothing using a rolling average
window_size = 7  # 7-day smoothing
for column, color, label in zip(columns_to_check, colors, labels):
    smoothed_percentages = pd.Series(null_percentages[column]).rolling(window=window_size, min_periods=1).mean()
    plt.plot(all_dates, smoothed_percentages, label=label, color=color)

# Update plot title, labels, and legend
plt.title("Missing Data Percentage (LST) Across All Years")
plt.xlabel('Date')
plt.ylabel('Missing Data Percentage (%)')
plt.legend()

# Force y-axis to always go from 0 to 100%
plt.ylim(0, 100)

# Save the plot
save_path = f"{save_dir}/nulls_percentage_all_years_lst.png"
plt.savefig(save_path)
plt.close()


Visualise null values for ERA.

In [ ]:
# Visualise null values for ERA by ID
# Define the columns to check for null values
columns_to_check = [
    'temperature_2m',
    'temperature_2m_min',
    'temperature_2m_max',
    'potential_evaporation_sum',
    'total_evaporation_sum',
    'total_precipitation_sum']

# Iterate over each year and create the plot
for year, group in era.groupby('Year'):
    # Initialize a plot for the current year
    plt.figure(figsize=(12, 6))
    
    # Initialize lists to store the null counts for each column
    null_counts = {col: [] for col in columns_to_check}
    id_2_values = []

    # Iterate over each unique 'ID_2' and calculate the null counts
    for id_2, sub_group in group.groupby('ID_2'):
        # For each column, count the null values for the current 'ID_2'
        for col in columns_to_check:
            null_counts[col].append(sub_group[col].isnull().sum())
        id_2_values.append(id_2)

    # Create the bar plot
    width = 0.15  # Bar width
    x = range(len(id_2_values))  # X positions for bars

    # Plot bars for each column of interest
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'brown']  # Colors for the bars
    for idx, col in enumerate(columns_to_check):
        plt.bar([p + width * idx for p in x], null_counts[col], width, label=col, color=colors[idx])

    # Add labels and title
    plt.xlabel('ID_2')
    plt.ylabel('Null Value Count')
    plt.title(f'Null Value Counts for Each ID_2 in {year}')
    plt.xticks([p + width * (len(columns_to_check) / 2 - 0.5) for p in x], id_2_values, rotation=45)
    plt.legend()

    # Save the plot for the current year
    save_path = f"{save_dir}/nulls_per_id_{year}_era.png"
    plt.tight_layout()  # Adjust layout to avoid overlap
    plt.savefig(save_path)
    plt.close()  # Close the plot to avoid displaying it inline


In [ ]:
columns_to_check = [
    'LST_Day_MEAN',
    'LST_Night_MEAN',
    'LST_Mean_MEAN']

# Create an empty dictionary to store plots
for year, group in lst.groupby('Year'):
    # Count the number of null values per day in each column
    null_count_day = group['LST_Day_MEAN'].isnull().groupby(group['Date']).sum()
    null_count_night = group['LST_Night_MEAN'].isnull().groupby(group['Date']).sum()
    null_count_mean = group['LST_Mean_MEAN'].isnull().groupby(group['Date']).sum()

    # Plotting the null values for each day of the year
    plt.figure(figsize=(10, 6))
    plt.plot(null_count_day.index, null_count_day, label='LST_Day_MEAN Nulls', color='red')
    plt.plot(null_count_night.index, null_count_night, label='LST_Night_MEAN Nulls', color='blue')
    plt.plot(null_count_mean.index, null_count_mean, label='LST_Mean_MEAN Nulls', color='green')

    # Add title and labels
    plt.title(f"Null Value Count (LST) for Each Day in {year}")
    plt.xlabel('Date')
    plt.ylabel('Null Value Count')
    plt.legend()

    # Save the plot for the year
    save_path = f"{save_dir}/nulls_per_day_{year}_lst.png"
    plt.savefig(save_path)
    plt.close()  # Close the plot to avoid displaying it inline


In [ ]:
save_dir = r"D:\Projects\TMU\tn_dengue\Graphs_2"

# Make sure 'Date' is a datetime type
era['Date'] = pd.to_datetime(lst['Date'])

# Extract the year from the 'Date' column
era['Year'] = era['Date'].dt.year

# Create an empty dictionary to store plots
for year, group in era.groupby('Year'):
    # Count the number of null values per day in each column
    null_count_day = group['LST_Day_MEAN'].isnull().groupby(group['Date']).sum()
    null_count_night = group['LST_Night_MEAN'].isnull().groupby(group['Date']).sum()
    null_count_mean = group['LST_Mean_MEAN'].isnull().groupby(group['Date']).sum()

    # Plotting the null values for each day of the year
    plt.figure(figsize=(10, 6))
    plt.plot(null_count_day.index, null_count_day, label='LST_Day_MEAN Nulls', color='red')
    plt.plot(null_count_night.index, null_count_night, label='LST_Night_MEAN Nulls', color='blue')
    plt.plot(null_count_mean.index, null_count_mean, label='LST_Mean_MEAN Nulls', color='green')

    # Add title and labels
    plt.title(f"Null Value Count for Each Day in {year}")
    plt.xlabel('Date')
    plt.ylabel('Null Value Count')
    plt.legend()

    # Save the plot for the year
    save_path = f"{save_dir}/nulls_per_day_{year}.png"
    plt.savefig(save_path)
    plt.close()  # Close the plot to avoid displaying it inline


In [ ]:
def rename_excel_sheets(excel_path):
    '''Function to load in whole excel sheet, load in as individual dataframes, and rename columns.'''
    excel_file = pd.ExcelFile(excel_path)
    sheet_names = excel_file.sheet_names
    
    if len(sheet_names) > 14:
        raise ValueError("Excel file has more than 14 sheets, cannot map to 2010-2023 range.")
    
    new_names = [f"in_{year}" for year in range(2010, 2010 + len(sheet_names))]
    
    dataframes = {new_name: excel_file.parse(sheet_name=old_name) for old_name, new_name in zip(sheet_names, new_names)}

    for idx, (name, df) in enumerate(dataframes.items()):
        year = 2010 + idx  # Compute the corresponding year dynamically

        rename_dict = {
            3: 'Infection_1', 4: 'Death_1',
            5: 'Infection_2', 6: 'Death_2',
            7: 'Infection_3', 8: 'Death_3',
            9: 'Infection_4', 10: 'Death_4',
            11: 'Infection_5', 12: 'Death_5',
            13: 'Infection_6', 14: 'Death_6',
            15: 'Infection_7', 16: 'Death_7',
            17: 'Infection_8', 18: 'Death_8',
            19: 'Infection_9', 20: 'Death_9',
            21: 'Infection_10', 22: 'Death_10',
            23: 'Infection_11', 24: 'Death_11',
            25: 'Infection_12', 26: 'Death_12',
            27: f'Total_Infections_{year}', 28: f'Total_Death_{year}', 29: f'Incidence_Rate_{year}', 31: f'Population_{year}' 
        }

        # Rename columns dynamically
        df.columns = [rename_dict[i] if i in rename_dict else col for i, col in enumerate(df.columns)]
        
        # Drop the first row (index 0)
        df.drop(index=0, inplace=True)
        
        # Create a global variable with the name corresponding to the sheet
        globals()[name] = df


In [ ]:
rename_excel_sheets(indonesia)

In [ ]:
'''In this cell we create GeoDataframe that contains both polygon information and also incidence data.'''

# Initialize an empty dictionary to store merged DataFrames
merged_all = {}

# List of DataFrame names (in_2010 to in_2023)
years = range(2010, 2024)

# Loop through each year, process the DataFrame, and store the result
for year in years:
    # Dynamically generate the DataFrame name (e.g., in_2010, in_2011, ..., in_2023)
    df_name = f'in_{year}'
    
    # Get the corresponding DataFrame (e.g., in_2010, in_2011)
    df = globals()[df_name]
    
    # Perform the merge with in_shp (shp remains the base)
    merged = pd.merge(in_shp, df, 
                      left_on=['NAME_1', 'NAME_2', 'ENGTYPE_2'], 
                      right_on=['Province', 'City', 'City/Regency'], 
                      how='left')

    # Store the merged DataFrame in the dictionary
    merged_all[year] = merged
    
    # Also store it as a global variable (merged_2010, merged_2011, etc.)
    globals()[f'merged_{year}'] = merged
    
    # Optional: Print the result to check the first few rows
    print(f'Merged data for {year}:')
    print(merged.head())

# Now, merged DataFrames exist globally AND in merged_all dictionary


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Ensure 'Date' is in datetime format
lst['Date'] = pd.to_datetime(lst['Date'])

# Extract the year from the 'Date' column
lst['Year'] = lst['Date'].dt.year

# Define the columns of interest
columns_to_check = ['LST_Day_MEAN', 'LST_Night_MEAN', 'LST_Mean_MEAN']



# Iterate over each year and create the plot for each column
for year, group in lst.groupby('Year'):
    null_counts_df = group.groupby('ID_2')[columns_to_check].apply(lambda x: x.isnull().sum()).reset_index()
    
    # Merge with shapefile
    merged = in_shp.merge(null_counts_df, on='ID_2', how='left')
    
    for col in columns_to_check:
        # Initialize a plot for the current year and column
        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
        merged.plot(column=col, cmap='OrRd', linewidth=0.8, edgecolor='black', legend=True, ax=ax)
        
        # Add labels and title
        plt.title(f'Choropleth Map of Null Counts for {col} in {year}')
        plt.axis('off')
        
        # Save the plot for the current year and column
        save_path = f"{save_dir}/choropleth_nulls_{year}_{col}.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()


In [ ]:
# Ensure 'Date' is in datetime format
era_1993_2023['Date'] = pd.to_datetime(era_1993_2023['Date'])
prec['Date'] = pd.to_datetime(prec['Date'])
# Extract the year from the 'Date' column
era_1993_2023['Year'] = era_1993_2023['Date'].dt.year
prec['Year'] = prec['Date'].dt.year


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os

# Merge dataframes on ID_2
merged = in_shp.merge(era_1993_2023, on='ID_2')

# # Ensure Date column is datetime type and extract Year
# merged['Date'] = pd.to_datetime(merged['Date'])
# merged['Year'] = merged['Date'].dt.year

# Select relevant statistic columns
stat_columns = [
    'temperature_2m_MEAN', 'temperature_2m_min_MEAN', 'temperature_2m_max_MEAN', 'potential_evaporation_sum_MEAN']

# Create output folder
os.makedirs("output_maps", exist_ok=True)

# Loop through each statistic
for stat in stat_columns:
    stat_name = stat.split('_Dropdown')[0]
    years = merged['Year'].unique()
    n_years = len(years)
    n_cols = 4
    n_rows = -(-n_years // n_cols)  # Ceiling division

    # Create a grid of subplots with 4 columns and rows as needed
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5 * n_rows))
    axes = axes.flatten()

    # Loop through each year and plot on the respective subplot
    for i, year in enumerate(years):
        yearly_data = merged[merged['Year'] == year]

        # Aggregate statistics by region (ID_2)
        summary = yearly_data.groupby('ID_2')[stat_columns].agg(['mean', 'std']).reset_index()

        # Flatten column names
        summary.columns = ['_'.join(col).strip() for col in summary.columns.values]

        # Merge back with shapefile
        mapped_data = in_shp.merge(summary, left_on='ID_2', right_on='ID_2_')

        metric_col = f"{stat}_mean"

        # Plotting the map on the respective subplot
        mapped_data.plot(
            column=metric_col,
            cmap='Blues',
            linewidth=0.8,
            ax=axes[i],
            edgecolor='0.8',
            legend=True
        )

        axes[i].set_title(f"{stat_name} (Mean) - {year}", fontsize=14)
        axes[i].axis('off')

    # Turn off any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    # Adjust layout and save figure
    plt.tight_layout()
    plt.savefig(f"output_maps/{stat_name}_Yearly_Grid.png", dpi=100)
    plt.close()

print("All statistic-based grid maps have been generated successfully!")


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os

# Assuming 'prec' is the new dataframe
# Merge dataframes on ID_2
merged = in_shp.merge(prec, on='ID_2')
regions = merged['Region'].unique()

# Select the relevant statistic column
stat_column = 'precipitation_MEAN'

# Create output folder
os.makedirs("output_maps", exist_ok=True)

for region in regions:
    # Filter data for the current region
    region_data = merged[merged['Region'] == region]
    region_shp = in_shp[in_shp['Region'] == region]
    
    # Get unique years for the region
    years = region_data['Year'].unique()
    n_years = len(years)
    n_cols = 4
    n_rows = -(-n_years // n_cols)  # Ceiling division

    # Create a grid of subplots with 4 columns and rows as needed
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5 * n_rows))
    axes = axes.flatten()

    # Loop through each year and plot on the respective subplot
    for i, year in enumerate(years):
        yearly_data = region_data[region_data['Year'] == year]

        # Aggregate statistics by region (ID_2)
        summary = yearly_data.groupby('ID_2')[stat_column].agg(['mean']).reset_index()

        # Flatten column names
        summary.columns = ['ID_2', f'{stat_column}_mean']

        # Merge back with region shapefile
        mapped_data = region_shp.merge(summary, on='ID_2')

        # Plotting the map on the respective subplot
        mapped_data.plot(
            column=f'{stat_column}_mean',
            cmap='Blues',
            linewidth=0.8,
            ax=axes[i],
            edgecolor='0.8',
            legend=True
        )

        axes[i].set_title(f"{region} - {stat_column.split('_')[0]} (Mean) - {year}", fontsize=14)
        axes[i].axis('off')

    # Turn off any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    # Adjust layout and save figure
    plt.tight_layout()
    plt.savefig(f"output_maps/{region}_{stat_column.split('_')[0]}_Yearly_Grid.png", dpi=100)
    plt.close()

print("All region-specific statistic-based grid maps have been generated successfully!")


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os

# Assuming 'prec' is the new dataframe
# Merge dataframes on ID_2
merged = in_shp.merge(prec, on='ID_2')

# Select the relevant statistic column
stat_column = 'precipitation_MEAN'

# Aggregate the values of the statistic for each region
region_aggregated = merged.groupby('Region')[stat_column].mean().reset_index()
region_aggregated.columns = ['Region', f'{stat_column}_mean']

# Merge the aggregated values back into the shapefile
in_shp_aggregated = in_shp.merge(region_aggregated, on='Region')

# Create output folder
os.makedirs("output_maps", exist_ok=True)

# Plotting the entire in_shp with aggregated values for each region
fig, ax = plt.subplots(figsize=(12, 12))
in_shp_aggregated.plot(
    column=f'{stat_column}_mean',
    cmap='Blues',
    linewidth=0.8,
    ax=ax,
    edgecolor='0.8',
    legend=True
)

ax.set_title(f"All Regions - {stat_column.split('_')[0]} (Mean) Aggregated", fontsize=14)
ax.axis('off')

# Save the plot
plt.tight_layout()
plt.savefig(f"output_maps/All_Regions_{stat_column.split('_')[0]}_Aggregated.png", dpi=100)
plt.close()

print("The aggregated map of all regions has been generated successfully!")


In [ ]:
# Regionwise maps
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os

# Merge dataframes on ID_2
merged = in_shp.merge(era_1993_2023, on='ID_2')
regions = merged['Region'].unique()

# Select relevant statistic columns
stat_columns = [
    'temperature_2m_MEAN', 'temperature_2m_min_MEAN', 'temperature_2m_max_MEAN', 'potential_evaporation_sum_MEAN']

# Create output folder
os.makedirs("output_maps", exist_ok=True)

for region in regions:
    # Filter data for the current region
    region_data = merged[merged['Region'] == region]
    region_shp = in_shp[in_shp['Region'] == region]
    
    for stat in stat_columns:
        stat_name = stat.split('_Dropdown')[0]
        years = region_data['Year'].unique()
        n_years = len(years)
        n_cols = 4
        n_rows = -(-n_years // n_cols)  # Ceiling division

        # Create a grid of subplots with 4 columns and rows as needed
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5 * n_rows))
        axes = axes.flatten()

        # Loop through each year and plot on the respective subplot
        for i, year in enumerate(years):
            yearly_data = region_data[region_data['Year'] == year]

            # Aggregate statistics by region (ID_2)
            summary = yearly_data.groupby('ID_2')[stat_columns].agg(['mean', 'std']).reset_index()

            # Flatten column names
            summary.columns = ['_'.join(col).strip() for col in summary.columns.values]

            # Merge back with region shapefile
            mapped_data = region_shp.merge(summary, left_on='ID_2', right_on='ID_2_')

            metric_col = f"{stat}_mean"

            # Plotting the map on the respective subplot
            mapped_data.plot(
                column=metric_col,
                cmap='Blues',
                linewidth=0.8,
                ax=axes[i],
                edgecolor='0.8',
                legend=True
            )

            axes[i].set_title(f"{region} - {stat_name} (Mean) - {year}", fontsize=14)
            axes[i].axis('off')

        # Turn off any unused subplots
        for j in range(i + 1, len(axes)):
            axes[j].axis('off')

        # Adjust layout and save figure
        plt.tight_layout()
        plt.savefig(f"output_maps/{region}_{stat_name}_Yearly_Grid.png", dpi=100)
        plt.close()

print("All region-specific statistic-based grid maps have been generated successfully!")


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os

# Assuming 'prec' is the new dataframe
# Merge dataframes on ID_2
merged = in_shp.merge(prec, on='ID_2')

# Select the relevant statistic column
stat_column = 'precipitation_MEAN'

# Create output folder
os.makedirs("output_maps", exist_ok=True)

# Get unique years
years = merged['Year'].unique()

for year in years:
    # Filter data for the current year
    year_data = merged[merged['Year'] == year]
    
    # Calculate the standard deviation for each region
    region_std = year_data.groupby('Region')[stat_column].std().reset_index()
    region_std.columns = ['Region', f'{stat_column}_std']

    # Merge the standard deviation values back into the shapefile
    in_shp_std = in_shp.merge(region_std, on='Region')

    # Plotting the entire in_shp with standard deviation values for each region
    fig, ax = plt.subplots(figsize=(12, 12))
    in_shp_std.plot(
        column=f'{stat_column}_std',
        cmap='Blues',
        linewidth=0.8,
        ax=ax,
        edgecolor='0.8',
        legend=True
    )

    ax.set_title(f"All Regions - {stat_column.split('_')[0]} Standard Deviation - {year}", fontsize=14)
    ax.axis('off')

    # Save the plot
    plt.tight_layout()
    plt.savefig(f"output_maps/All_Regions_{stat_column.split('_')[0]}_Std_{year}.png", dpi=100)
    plt.close()

print("Standard deviation maps for all regions and years have been generated successfully!")


In [ ]:
# Mean of every polygon in a given region, then plot whole region as one
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os

# Merge dataframes on ID_2
merged = in_shp.merge(era_1993_2023, on='ID_2')

# Extract unique regions from the merged dataframe
regions = merged['Region'].unique()

# Select relevant statistic columns
stat_columns = [
    'temperature_2m_MEAN', 'temperature_2m_min_MEAN', 'temperature_2m_max_MEAN', 'potential_evaporation_sum_MEAN']

# Create output folder
os.makedirs("output_maps", exist_ok=True)

# Loop through each statistic
for stat in stat_columns:
    stat_name = stat.split('_Dropdown')[0]
    years = merged['Year'].unique()
    n_years = len(years)
    n_cols = 4
    n_rows = -(-n_years // n_cols)  # Ceiling division

    # Create a grid of subplots with 4 columns and rows as needed
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5 * n_rows))
    axes = axes.flatten()

    # Loop through each year and plot on the respective subplot
    for i, year in enumerate(years):
        yearly_data = merged[merged['Year'] == year]

        # Aggregate statistics by Region (calculate mean of all polygons per region)
        region_summary = yearly_data.groupby('Region')[stat_columns].mean().reset_index()

        # Merge back with the full shapefile
        mapped_data = in_shp.merge(region_summary, on='Region')

        metric_col = f"{stat}"

        # Plot the whole map with each region as one unit
        mapped_data.plot(
            column=metric_col,
            cmap='Blues',
            linewidth=0.15,
            ax=axes[i],
            edgecolor='0.8',
            legend=True
        )

        axes[i].set_title(f"{stat_name} (Region Mean) - {year}", fontsize=14)
        axes[i].axis('off')

    # Turn off any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    # Adjust layout and save figure
    plt.tight_layout()
    plt.savefig(f"output_maps/{stat_name}_Region_Aggregated_Yearly_Grid.png", dpi=100)
    plt.close()

print("All region-aggregated statistic-based grid maps have been generated successfully!")


In [ ]:
# Merge dataframes on ID_2
merged = in_shp.merge(prec, on='ID_2')

# Ensure Date column is datetime type and extract Year
merged['Date'] = pd.to_datetime(merged['Date'])
merged['Year'] = merged['Date'].dt.year

# Define the relevant column
stat_column = 'precipitation_MEAN'

# Create output folder
os.makedirs("output_maps", exist_ok=True)

# Loop through each year
for year in merged['Year'].unique():
    yearly_data = merged[merged['Year'] == year]

    # Aggregate statistics by region (ID_2)
    summary = yearly_data.groupby('ID_2')[stat_column].agg(['mean', 'std']).reset_index()

    # Rename columns for clarity
    summary.rename(columns={'mean': 'precipitation_mean', 'std': 'precipitation_std'}, inplace=True)

    # Merge back with shapefile
    mapped_data = in_shp.merge(summary, on='ID_2')

    # Generate plots for mean and standard deviation
    for metric in ['mean', 'std']:
        metric_col = f"precipitation_{metric}"
        title_metric = "Mean" if metric == "mean" else "Standard Deviation"

        fig, ax = plt.subplots(1, 1, figsize=(10, 8))
        mapped_data.plot(column=metric_col, cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
        ax.set_title(f"Precipitation {title_metric} - {year}", fontsize=15)
        ax.axis('off')

        # Save figure
        plt.savefig(f"output_maps/precipitation_{metric}_{year}.png", dpi=100)
        plt.close()

print("All maps have been generated successfully!")
